In [ ]:
%pip install oracledb

In [ ]:
from oracledb import connect


class TakDBManager:
    def makeConCur(url):
        con = connect(url)
        cur = con.cursor()
        return con, cur
    
    def closeConCur(con, cur) :
        cur.close()
        con.close()
        

In [ ]:
import oracledb
import json

def insert_strategy_from_file(file_path, db_url):
    """JSON 파일을 읽어 Oracle DB에 레이스 전략 정보를 저장합니다."""
    
    con, cur = None, None
    try:
        # JSON 파일 읽기
        with open(file_path, 'r') as f:
            data = json.load(f)

        # DB 연결
        con, cur = TakDBManager.makeConCur(db_url)

        # 1. RACE_STRATEGIES 테이블에 마스터 데이터 삽입
        sql_strategy = """
            INSERT INTO RACE_STRATEGIES (
                track_name, starting_compound, total_laps, 
                estimated_race_time, total_pit_stops, strategy_summary
            ) VALUES (
                :track_name, :starting_compound, :total_laps, 
                :estimated_race_time, :total_pit_stops, :strategy_summary
            ) RETURNING strategy_id INTO :strategy_id
        """
        
        # RETURNING INTO를 사용하기 위한 변수 생성
        new_strategy_id = cur.var(oracledb.DB_TYPE_NUMBER)
        
        # 시작 컴파운드 정보 추출 (첫 번째 스틴트의 컴파운드)
        start_compound = data['stints'][0]['compound'] if data['stints'] else None

        cur.execute(sql_strategy, 
            track_name=data['track_name'],
            starting_compound=start_compound,
            total_laps=data['total_laps'],
            estimated_race_time=data['estimated_race_time_seconds'],
            total_pit_stops=data['total_pit_stops'],
            strategy_summary=data['strategy_summary'],
            strategy_id=new_strategy_id  # RETURNING 값을 받을 변수
        )
        
        # 방금 생성된 strategy_id 가져오기
        inserted_id = new_strategy_id.getvalue()[0]
        print(f"'{data['track_name']}' 전략 데이터 삽입 성공. Strategy ID: {inserted_id}")

        # 2. STRATEGY_STINTS 테이블에 스틴트 상세 데이터 반복 삽입
        sql_stint = """
            INSERT INTO STRATEGY_STINTS (
                strategy_id, stint_number, compound, 
                start_lap, end_lap, stint_length
            ) VALUES (
                :strategy_id, :stint_number, :compound, 
                :start_lap, :end_lap, :stint_length
            )
        """
        
        # executemany를 사용하기 위해 데이터 리스트 생성
        stints_data = []
        for stint in data['stints']:
            stints_data.append({
                "strategy_id": inserted_id, # 위에서 받은 ID 사용
                "stint_number": stint['stint_number'],
                "compound": stint['compound'],
                "start_lap": stint['start_lap'],
                "end_lap": stint['end_lap'],
                "stint_length": stint['stint_length']
            })
        
        cur.executemany(sql_stint, stints_data)
        print(f"{len(stints_data)}개의 스틴트 데이터 삽입 성공.")

        # 모든 작업이 성공했으면 DB에 최종 반영
        con.commit()

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        if con:
            con.rollback() # 오류 발생 시 모든 작업을 취소
    finally:
        if con and cur:
            TakDBManager.closeConCur(con, cur)
            print("DB 연결을 종료합니다.")


In [ ]:
db_url = "track/track@9.234.242.124:1521/xepdb1"
    
track = [
    "Abu_Dhabi",
    "Australian",
    "Austrian",
    "Azerbaijan",
    "Bahrain",
    "Belgian",
    "British",
    "Canadian",
    "Chinese",
    "Dutch",
    "Eifel",
    "Emilia_Romagna",
    "French",
    "German",
    "Hungarian",
    "Italian",
    "Japanese",
    "Las_Vegas",
    "Mexico_City",
    "Miami",
    "Monaco",
    "Portuguese",
    "Qatar",
    "Russian",
    "Sakhir",
    "Saudi_Arabian",
    "Singapore",
    "Spanish",
    "Styrian",
    "Sao_Paulo",
    "Turkish",
    "Tuscan",
    "United_States"
]
compound = ["HARD", "MEDIUM", "SOFT"]
for tname in track :
    for cname in compound:
        file = f"/home/azureuser/cloudfiles/code/Users/student01_27/project/soldesk_notebook/json_result/result/{tname}_{cname}_strategy_result.json"
        print(f"\n--- {file} 파일 처리 시작 ---")
        insert_strategy_from_file(file, db_url)

In [1]:
import oracledb
import json  # JSON을 다루기 위한 라이브러리 import

# TakDBManager 클래스는 이전에 정의된 것과 동일하다고 가정합니다.
class TakDBManager:
    @staticmethod
    def makeConCur(url):
        con = oracledb.connect(url)
        cur = con.cursor()
        return con, cur
    
    @staticmethod
    def closeConCur(con, cur) :
        cur.close()
        con.close()

def get_strategy_as_json(track_name, starting_compound, db_url):
    """트랙 이름과 시작 컴파운드를 받아 레이스 전략을 조회하고, 결과를 JSON 형식으로 반환합니다."""
    
    con, cur = None, None
    try:
        con, cur = TakDBManager.makeConCur(db_url)
        
        # 조회에 필요한 모든 컬럼을 가져옵니다.
        sql = """
            SELECT
                rs.total_laps,
                rs.estimated_race_time,
                rs.total_pit_stops,
                rs.strategy_summary,
                ss.stint_number,
                ss.compound,
                ss.start_lap,
                ss.end_lap,
                ss.stint_length
            FROM
                RACE_STRATEGIES rs
            JOIN
                STRATEGY_STINTS ss ON rs.strategy_id = ss.strategy_id
            WHERE
                rs.track_name = :t_name 
                AND rs.starting_compound = :s_compound
            ORDER BY
                ss.stint_number
        """
        
        cur.execute(sql, t_name=track_name, s_compound=starting_compound)
        results = cur.fetchall()
        
        if not results:
            # 결과가 없으면 None을 반환
            return None

        # --- 결과를 JSON 구조(파이썬 딕셔너리)로 재구성 ---
        
        # 1. 최상위 정보 구성 (첫 번째 결과 행에서 가져옴)
        first_row = results[0]
        strategy_dict = {
            "track_name": track_name,
            "total_laps": first_row[0],
            "estimated_race_time_seconds": first_row[1],
            # 분 단위 시간은 초 단위에서 계산 (필요시)
            "estimated_race_time_minutes": round(first_row[1] / 60, 2), 
            "total_pit_stops": first_row[2],
            "strategy_summary": first_row[3],
            "stints": []  # 스틴트 정보를 담을 빈 리스트 생성
        }
        
        # 2. 스틴트 상세 정보 구성 (모든 결과 행을 반복 처리)
        for row in results:
            stint_dict = {
                "stint_number": row[4],
                "compound": row[5],
                "start_lap": row[6],
                "end_lap": row[7],
                "stint_length": row[8]
            }
            strategy_dict["stints"].append(stint_dict)
            
        # 완성된 파이썬 딕셔너리를 반환
        return strategy_dict

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        return None # 오류 발생 시에도 None 반환
    finally:
        if con and cur:
            TakDBManager.closeConCur(con, cur)

# --- 실행 ---
if __name__ == "__main__":
    db_url = "track/track@9.234.242.124:1521/xepdb1"
    
    target_track = "Bahrain"
    target_compound = "SOFT"
    
    # 함수를 호출하여 결과를 파이썬 딕셔너리로 받음
    strategy_result = get_strategy_as_json(target_track, target_compound, db_url)
    
    if strategy_result:
        # 받은 딕셔너리를 보기 좋은 JSON 문자열로 변환하여 출력
        # indent=2 : 보기 좋게 2칸 들여쓰기
        # ensure_ascii=False : 한글이 깨지지 않도록 설정
        json_output = json.dumps(strategy_result, indent=2, ensure_ascii=False)
        
        print(json_output)
    else:
        print(f"'{target_track}' 트랙의 '{target_compound}' 시작 전략을 찾을 수 없습니다.")

{
  "track_name": "Bahrain",
  "total_laps": 57,
  "estimated_race_time_seconds": 6542.4326,
  "estimated_race_time_minutes": 109.04,
  "total_pit_stops": 2,
  "strategy_summary": "Start with SOFT -> Lap 11 PIT to HARD -> Lap 40 PIT to HARD -> Finish",
  "stints": [
    {
      "stint_number": 1,
      "compound": "SOFT",
      "start_lap": 1,
      "end_lap": 11,
      "stint_length": 11
    },
    {
      "stint_number": 2,
      "compound": "HARD",
      "start_lap": 12,
      "end_lap": 40,
      "stint_length": 29
    },
    {
      "stint_number": 3,
      "compound": "HARD",
      "start_lap": 41,
      "end_lap": 57,
      "stint_length": 17
    }
  ]
}
